In [2]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd

In [3]:
import os
from pyspark import SparkFiles
import zipfile
import io
from config import FTP_DATASOURCE, FTP_DATASOURCE_URI

In [4]:
# inicialize session and get context spark
spark = SparkSession.builder.appName("ftp_spark").getOrCreate()
sc = spark.sparkContext

In [31]:
#function to extract and save database path
def zip_extract_save(x):
    in_memory_data = io.BytesIO(x[1])
    with zipfile.ZipFile(in_memory_data, "r") as zipped:
            zipped.extractall("database")
    
zips = sc.binaryFiles(FTP_DATASOURCE_URI)
files_data = zips.map(zip_extract_save)
files_data.collect()

In [55]:
#function unzip to rdd
def zip_extract_rdd(x):
    in_memory_data = io.BytesIO(x[1])
    
    zf = zipfile.ZipFile(in_memory_data)
    f = zf.read('data_municipio.csv').decode('UTF-8')
    seq = 0
    all_lines = []
    for line in f.split('\r'):
        seq+=1
        if(seq>1 and line !='\n'):
            l = line.split(';')
            all_lines.append(tuple(l[0:4]))
    return all_lines


In [ ]:
#consumindo ftp e get arquivo binário
zips = sc.binaryFiles(FTP_DATASOURCE_URI)

In [ ]:
#extracao e transformacao em rdd
files_data = zips.map(zip_extract_rdd)
schema = StructType([ \
    StructField("transacao",StringType(),True), \
    StructField("municipio",StringType(),True), \
    StructField("estado",StringType(),True), \
    StructField("data_atualizacao", StringType(), True),
  ])
#criando dataframe com o rdd e schema acima
df = spark.createDataFrame(files_data.collect()[0],schema)

In [56]:
df.show()

+---------+--------------+------+----------------+
|transacao|     municipio|estado|data_atualizacao|
+---------+--------------+------+----------------+
|       10|    UBERLANDIA|    MG|      23/02/2021|
|       50|     SAO PAULO|    SP|      24/02/2021|
|        5|     SAO PAULO|    SP|      25/02/2021|
|       20|    UBERLANDIA|    MG|      23/02/2021|
|      100|RIO DE JANEIRO|    RJ|      23/02/2021|
|    200,2|RIO DE JANEIRO|    RJ|      24/02/2021|
|    50,41|      CAMPINAS|    SP|      24/02/2021|
|       30|      CAMPINAS|    SP|      24/02/2021|
|      500|     SAO PAULO|    SP|      25/02/2021|
+---------+--------------+------+----------------+



In [54]:
#excrita em parquet
df.write.format('parquet').save('database/data_municipio_zip_2_df')

In [ ]:
#Lendo arquivo que foi salvo e reescrevendo

In [56]:
df = spark.read.csv('database/data_municipio.csv', sep=';',header=True, ignoreTrailingWhiteSpace=True)
df = df.drop('_c4')

In [82]:
# savando como parquet0
df.repartition(8).write.format('parquet').save('database/data_municipio.parquet')